## 导入必要的库

In [1]:
import torch

## 定义 NCHW 格式 (Batch, Channel, Height, Width)

In [2]:
N, C, H, W = 1, 3, 4, 4
x_nchw = torch.randn(N, C, H, W)

print("=== NCHW (默认) ===")
print(f"Shape: {x_nchw.shape}")
print(f"Stride: {x_nchw.stride()}")

=== NCHW (默认) ===
Shape: torch.Size([1, 3, 4, 4])
Stride: (48, 16, 4, 1)


### NCHW Stride 解释

典型 stride: (48, 16, 4, 1) -> 这里的 16 是 H*W

意味着：在内存中，同一个 Channel 的像素是聚在一起的。要跨越一个 Channel (从 C0 到 C1)，需要跳过 H*W (16) 个元素。

## 转换为 Channels Last (NHWC)

注意：这通常会触发物理重排，以适应新的 Stride 策略

In [3]:
x_nhwc = x_nchw.to(memory_format=torch.channels_last)

print("\n=== NHWC (Channels Last) ===")
print(f"Shape: {x_nhwc.shape}")  # Shape 看起来没变！依然是 (N, C, H, W)
print(f"Stride: {x_nhwc.stride()}")


=== NHWC (Channels Last) ===
Shape: torch.Size([1, 3, 4, 4])
Stride: (48, 1, 12, 3)


### NHWC Stride 解释

典型 stride: (48, 1, 12, 3) -> 注意第二个元素变成了 1！

意味着：Stride[1] (Channel 维度) 是 1。说明：在内存中，C0, C1, C2 是紧挨着的 (像素的 RGB 值在一起)。

## 验证硬件亲和性（逻辑验证）

模拟访问一个像素点在内存中的连续性

In [4]:
print("\n>>> 访问同一个像素的不同通道 (x[0, :, 0, 0]):")

# 获取物理偏移量 helper
def get_offset(tensor, n, c, h, w):
    s = tensor.stride()
    return n * s[0] + c * s[1] + h * s[2] + w * s[3]

offset_c0 = get_offset(x_nhwc, 0, 0, 0, 0)
offset_c1 = get_offset(x_nhwc, 0, 1, 0, 0)
offset_c2 = get_offset(x_nhwc, 0, 2, 0, 0)

print(f"C0 偏移量: {offset_c0}")
print(f"C1 偏移量: {offset_c1}")
print(f"C2 偏移量: {offset_c2}")
print("结论: 在 Channels Last 格式中，不同通道的像素值在内存中是紧邻的 (距离为1)。")


>>> 访问同一个像素的不同通道 (x[0, :, 0, 0]):
C0 偏移量: 0
C1 偏移量: 1
C2 偏移量: 2
结论: 在 Channels Last 格式中，不同通道的像素值在内存中是紧邻的 (距离为1)。


## 关键理解

### 抽象的胜利

用户看到的 shape 永远是 (N, C, H, W)，符合直觉。

### 底层的变换

- **NCHW**: 内存排布是 RRRR...GGGG...BBBB...。这适合做空间卷积吗？不一定。
- **NHWC**: 内存排布是 RGBRGBRGB...。

### 为什么快

现代 GPU 的计算单元（如 Tensor Core）通常设计为一次处理向量化的点积（比如一次读取 32 个数值）。如果是 NHWC，GPU 可以一次性把一个像素的所有通道（比如 64 或 128 个通道）读入缓存行（Cache Line）进行计算，这是极高效率的访存模式。